# Q-Learning in Windy Gridworld Environment

In [1]:
import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
from collections import defaultdict
from gym.envs.toy_text import discrete

In [2]:
UP, RIGHT, DOWN, LEFT = 0, 1, 2, 3


class WindyGridworldEnv(discrete.DiscreteEnv):
    def __init__(self):
        self.shape = (7, 10)
        nS = self.shape[0] * self.shape[1]
        nA = 4

        winds = np.zeros(self.shape)
        winds[:, (3, 4, 5, 8)] = 1
        winds[:, (6, 7)] = 2

        self.goal = (3, 7)

        isd = np.zeros(nS)
        isd[np.ravel_multi_index((3, 0), self.shape)] = 1.0

        P = {}
        for s in range(nS):
            position = np.unravel_index(s, self.shape)
            P[s] = {a: [] for a in range(nA)}
            P[s][UP] = self._calc_transition_probabilities(
                position, [-1, 0], winds
            )  # -1 => Moves row up, 0 => Same Col
            P[s][RIGHT] = self._calc_transition_probabilities(
                position, [0, 1], winds
            )  # -1 => Moves row up, 0 => Same Col
            P[s][DOWN] = self._calc_transition_probabilities(
                position, [1, 0], winds
            )  # -1 => Moves row up, 0 => Same Col
            P[s][LEFT] = self._calc_transition_probabilities(
                position, [0, -1], winds
            )  # -1 => Moves row up, 0 => Same Col

        super(WindyGridworldEnv, self).__init__(nS, nA, P, isd)

    def _calc_transition_probabilities(self, pos, delta, winds):
        new_pos = (
            np.array(pos) + np.array(delta) + np.array([-1, 0]) * winds[tuple(pos)]
        )  # isn't pos already a tuple??
        new_pos = self._limit_coordinates(new_pos).astype(int)
        new_state = np.ravel_multi_index(tuple(new_pos), self.shape)

        is_done = tuple(new_pos) == self.goal

        return [(1.0, new_state, -1.0, is_done)] # prob, new_stae, reward, done

    def _limit_coordinates(self, coord):
        return np.clip(coord, (0, 0), np.array(self.shape) - 1)

    def render(self, mode="human"):
        outfile = sys.stdout

        for s in range(self.nS):
            pos = np.unravel_index(s, self.shape)

            if self.s == s:
                output = " s "
            elif pos == self.goal:
                output = " G "

            else:
                output = " _ "

            if pos[1] == 0:
                output = output.lstrip()
            if pos[1] == self.shape[1] - 1:
                output = output.rstrip()
                output += "\n"

            outfile.write(output)
        outfile.write("\n")


In [3]:
env = WindyGridworldEnv()

In [4]:
env.reset()
env.render()

_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
s  _  _  _  _  _  _  G  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _



In [5]:
print(env.step(1))
env.render()

(31, -1.0, False, {'prob': 1.0})
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
_  s  _  _  _  _  _  G  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _
_  _  _  _  _  _  _  _  _  _



# It is Q-Learning Time! 

In [6]:
def epsilon_greedy_policy(Q, state, nA, epsilon):
    probs = np.ones(nA) * epsilon / nA # should this not be nA - 1?

    best_action = np.argmax(Q[state])
    probs[best_action] += 1 - epsilon

    return probs

In [7]:
def Q_learning(env: gym.Env, episodes: int, learning_rate: float, gamma: float, epsilon: float):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    x = np.arange(episodes)
    y = np.zeros(episodes)

    for ep in range(episodes):
        state = env.reset()

        for step in range(int(1e5)):
            probs = epsilon_greedy_policy(Q, state, env.action_space.n, epsilon)
            action = np.random.choice(np.arange(env.action_space.n), p=probs)

            next_state, reward, done, _  = env.step(action)

            Q[state][action] += learning_rate * ((reward + gamma * np.max(Q[next_state])) - Q[state][action])

            if done:
                y[ep] = step
                break
        
            state = next_state

    return x, y, Q


In [8]:
# x, y, Q = Q_learning(env, 1000, 0.01, .99, .05)

In [9]:
# plt.figure(figsize=(18, 5))
# plt.plot(x, y)

# plt.xlabel("episodes")
# plt.ylabel("steps required")

In [10]:
# state, done = env.reset(), False
# env.render()
# rewards = []

# while not done:
#     action = np.argmax(Q[state])
#     # print(f"Taking Action: {action}")
#     # actions.append(action)
#     state, reward, done, _ = env.step(action)
#     # env.render()

#     rewards.append(reward)


In [52]:
class Agent:
    def __init__(
        self, lr, gamma, n_actions, n_states, eps_start, eps_end, eps_dec
    ) -> None:
        self.lr = lr
        self.gamma = gamma
        self.n_actions = n_actions
        self.n_states = n_states
        self.epsilon = eps_start
        self.eps_end = eps_end
        self.eps_dec = eps_dec

        self.Q = {s: np.random.randn(self.n_actions) for s in range(self.n_states)}

        self.total_steps = 0

    def _epsilon_greedy_policy(self, state):
        probs = (
            np.ones(self.n_actions) * self.epsilon / self.n_actions
        )  # should this not be nA - 1?

        best_action = np.argmax(self.Q[state])
        probs[best_action] += 1 - self.epsilon

        return probs

    def choose_action(self, state):
        probs = self._epsilon_greedy_policy(state)
        return np.random.choice(np.arange(self.n_actions), p=probs)

    def eps_udpate(self):
        self.epsilon = max(self.epsilon - self.eps_dec, self.eps_end)

    def learn(self, state, action, reward, state_):
        self.total_steps += 1
        self.Q[state][action] += self.lr * (
            (reward + self.gamma * np.amax(self.Q[state_])) - self.Q[state][action]
        )

        self.eps_udpate()


In [54]:
from tqdm import tqdm


env = gym.make("FrozenLake-v1")
agent = Agent(
    lr=0.001,
    gamma=0.99,
    n_actions=env.action_space.n,
    n_states=env.observation_space.n,
    eps_start=0.1,
    eps_end=0.01,
    eps_dec=1e-3,
)

ngames = int(5e4)
win_pct = []
scores = []

for i in tqdm(range(ngames)):
    state, done = env.reset(), False
    score = 0

    while not done:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state)

        if agent.total_steps % 1000 == 0:
            print(f"Q[{state}]: {agent.Q[state]}")

        score += reward
        state = next_state

    scores.append(score)
    if i % 100 == 0:
        average = np.mean(scores[-100:])
        win_pct.append(average)


plt.figure(figsize=(18, 5))
plt.plot(scores[::100])
plt.plot(win_pct)
plt.legend(["Scores", "Win Percentages"])
plt.show()


  1%|          | 503/50000 [00:00<00:19, 2534.13it/s]

Q[8]: [-1.05641711 -1.25800186 -0.66419868  0.27465538]
Q[4]: [-0.85861905  0.17171838 -0.79147759 -0.96480823]
Q[9]: [-1.105696   -0.101317    1.08191194  0.63386408]
Q[6]: [-0.29655543  0.38539145  2.03970749  0.05484432]


  3%|▎         | 1306/50000 [00:00<00:18, 2572.70it/s]

Q[3]: [-0.073238   -0.99465814  1.48234467 -0.60721787]
Q[4]: [-0.85741122  0.43183953 -0.788412   -0.95997756]
Q[4]: [-0.85611448  0.47713673 -0.78644296 -0.95997756]
Q[0]: [-0.31364068  0.94795345 -0.00097085 -0.93631903]


  4%|▎         | 1830/50000 [00:00<00:18, 2575.06it/s]

...

 24%|██▍       | 11903/50000 [00:04<00:14, 2555.83it/s]

Q[4]: [-0.79617863  1.14372756 -0.72545684 -0.89522903]
Q[1]: [ 1.12147526  0.22891609 -0.59146694  0.20294669]
Q[4]: [-0.79041326  1.14665998 -0.72358892 -0.88909156]
Q[0]: [-0.25075621  1.11129836  0.06265499 -0.85350622]


 25%|██▍       | 12425/50000 [00:04<00:14, 2522.46it/s]

Q[1]: [ 1.12634966  0.22891609 -0.58235294  0.20406021]
Q[8]: [-1.00908289 -1.20869633 -0.63906098  1.15945636]


KeyboardInterrupt: 